In [1]:
from robustness_analysis.DiGraph import DiGraph
from robustness_analysis.Metaweb import Metaweb
import robustness_analysis.Constants as Constants
from robustness_analysis.Enums import Preparation, AttackStrategy
from robustness_analysis.Perturbation import Perturbation
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
mw = Metaweb(Constants.FOODWEB_01)
mw.prepare(preparation=Preparation.USE_AS_IS)
edge_df = mw.get_edge_df()
graph = DiGraph(edge_df, source_col=Constants.SOURCE_COL, target_col=Constants.TARGET_COL)

In [3]:
perturbation = Perturbation(graph, AttackStrategy.RANDOM)

perturbation.run()

results = perturbation.get_results()

In [15]:
density = results["density"]
average_in_degree = results["average_in_degree"]

# Create two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 9), dpi=1000)

# Plotting Density on the first (top) plot
ax1.plot(density, color='royalblue', linestyle='-', linewidth=2)
ax1.set_ylabel('Density', fontsize=14)
ax1.set_title('Density Robustness Trend', fontsize=16)
ax1.grid(True, which='both', linestyle='--', linewidth=0.5)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_linewidth(0.5)
ax1.spines['left'].set_linewidth(0.5)

# Plotting Average In-Degree on the second (bottom) plot
ax2.plot(average_in_degree, color='darkorange', linestyle='-', linewidth=2)
ax2.set_xlabel('Primary Extinctions', fontsize=14)
ax2.set_ylabel('Average In-Degree', fontsize=14)
ax2.set_title('Average In-Degree Robustness Trend', fontsize=16)
ax2.grid(True, which='both', linestyle='--', linewidth=0.5)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['bottom'].set_linewidth(0.5)
ax2.spines['left'].set_linewidth(0.5)

# Set main title
fig.suptitle('Attack Strategy: Random', fontsize=18, y=1.05)

plt.tight_layout()  # Adjust subplot parameters for better layout

# Finally, show the plot
plt.show()
